# Day14_0: 군집화 (Clustering) - 정답 노트북

이 노트북은 Day14_0 실습 퀴즈의 정답과 상세 풀이를 제공합니다.

---

## 환경 설정

In [ ]:
# 필수 라이브러리 임포트
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples

import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

print("환경 설정 완료!")

In [ ]:
# 데이터 준비 (문제 노트북과 동일)
np.random.seed(42)
n_customers = 500

group1 = np.random.multivariate_normal([30, 20], [[50, 10], [10, 100]], 125)
group2 = np.random.multivariate_normal([30, 80], [[50, 10], [10, 100]], 125)
group3 = np.random.multivariate_normal([80, 20], [[50, 10], [10, 100]], 125)
group4 = np.random.multivariate_normal([80, 80], [[50, 10], [10, 100]], 125)

customer_data = np.vstack([group1, group2, group3, group4])
customer_data = np.clip(customer_data, 1, 100)

df_customers = pd.DataFrame({
    'CustomerID': range(1, n_customers + 1),
    'Annual_Income': customer_data[:, 0].astype(int),
    'Spending_Score': customer_data[:, 1].astype(int)
})
df_customers['Age'] = np.random.randint(18, 70, n_customers)
df_customers['Visit_Frequency'] = np.random.randint(1, 30, n_customers)

X = df_customers[['Annual_Income', 'Spending_Score']].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("데이터 준비 완료!")
print(f"데이터 크기: {X_scaled.shape}")

---

## Q1. K-Means 기본 ⭐

**문제**: K=3으로 K-Means를 수행하고 각 군집의 샘플 수를 출력하세요.

In [ ]:
# 정답 코드
from sklearn.cluster import KMeans

# K=3으로 K-Means 모델 생성 및 학습
kmeans_q1 = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_q1 = kmeans_q1.fit_predict(X_scaled)

# 각 군집의 샘플 수 출력
print("=== K=3 K-Means 군집별 샘플 수 ===")
for cluster in range(3):
    count = (labels_q1 == cluster).sum()
    print(f"군집 {cluster}: {count}개")

In [ ]:
# 테스트/검증
assert len(set(labels_q1)) == 3, "군집 수가 3이어야 합니다"
assert len(labels_q1) == 500, "모든 샘플이 할당되어야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- KMeans(n_clusters=3)으로 3개 군집 지정
- fit_predict()로 학습과 레이블 예측 동시 수행
- 레이블 배열에서 각 값의 개수 카운트

**핵심 개념**:
- n_clusters: 생성할 군집 수
- random_state: 재현성을 위한 시드
- n_init: 초기화 반복 횟수 (더 안정적인 결과)

**대안**:
```python
# pd.Series로 카운트
pd.Series(labels_q1).value_counts().sort_index()

# np.unique로 카운트
unique, counts = np.unique(labels_q1, return_counts=True)
dict(zip(unique, counts))
```

**흔한 실수**:
- 스케일링 안 함 (거리 기반이므로 필수)
- random_state 미지정 (결과 재현 불가)

---

## Q2. Inertia 계산 ⭐

**문제**: K=5인 K-Means의 inertia 값을 출력하세요.

In [ ]:
# 정답 코드
# K=5로 K-Means 모델 생성 및 학습
kmeans_q2 = KMeans(n_clusters=5, random_state=42, n_init=10)
kmeans_q2.fit(X_scaled)

# Inertia 출력
print(f"K=5 K-Means Inertia: {kmeans_q2.inertia_:.2f}")

In [ ]:
# 테스트/검증
assert hasattr(kmeans_q2, 'inertia_'), "모델이 학습되어야 합니다"
assert kmeans_q2.inertia_ > 0, "Inertia는 양수여야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- K-Means 모델 학습 후 inertia_ 속성 접근
- Inertia = 각 샘플과 해당 군집 중심점 간 거리의 제곱합

**핵심 개념**:
- Inertia (Within-Cluster Sum of Squares)
- 값이 작을수록 군집이 조밀함
- K가 증가하면 항상 감소 (과적합 주의)

**실무 팁**:
- Inertia 단독으로는 최적 K 결정 어려움
- Elbow Method와 함께 사용

---

## Q3. Silhouette Score ⭐⭐

**문제**: K=4인 K-Means의 Silhouette Score를 계산하세요.

In [ ]:
# 정답 코드
from sklearn.metrics import silhouette_score

# K=4로 K-Means 수행
kmeans_q3 = KMeans(n_clusters=4, random_state=42, n_init=10)
labels_q3 = kmeans_q3.fit_predict(X_scaled)

# Silhouette Score 계산
sil_score_q3 = silhouette_score(X_scaled, labels_q3)

print(f"K=4 K-Means Silhouette Score: {sil_score_q3:.4f}")

In [ ]:
# 테스트/검증
assert -1 <= sil_score_q3 <= 1, "Silhouette Score는 -1~1 범위"
assert sil_score_q3 > 0.3, "좋은 군집화는 0.3 이상"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- 군집화 수행 후 silhouette_score() 함수 사용
- 입력: 스케일링된 데이터와 레이블

**핵심 개념**:
- Silhouette Score = (b - a) / max(a, b)
- a: 같은 군집 내 평균 거리
- b: 가장 가까운 다른 군집과의 평균 거리
- 범위: -1 ~ 1 (1에 가까울수록 좋음)

**해석 기준**:
- 0.7~1.0: 강한 구조
- 0.5~0.7: 합리적 구조
- 0.25~0.5: 약한 구조
- < 0.25: 구조 없음 또는 잘못된 군집

---

## Q4. Elbow Plot 그리기 ⭐⭐

**문제**: K=1~10에 대해 inertia를 계산하고 px.line()으로 Elbow Plot을 그리세요.

In [ ]:
# 정답 코드
# K=1~10에 대해 Inertia 계산
K_range = range(1, 11)
inertias = []

for k in K_range:
    kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_temp.fit(X_scaled)
    inertias.append(kmeans_temp.inertia_)

# DataFrame으로 정리
elbow_df = pd.DataFrame({
    'K': list(K_range),
    'Inertia': inertias
})

# Elbow Plot
fig = px.line(elbow_df, x='K', y='Inertia', markers=True,
              title='Elbow Method: 최적 군집 수 찾기')
fig.update_layout(xaxis_title='군집 수 (K)', yaxis_title='Inertia')
fig.show()

In [ ]:
# 테스트/검증
assert len(inertias) == 10, "K=1~10 모두 계산해야 합니다"
assert inertias[0] > inertias[-1], "K가 증가하면 Inertia는 감소"
print("테스트 통과!")
print(f"\nK별 Inertia:\n{elbow_df}")

### 💡 풀이 설명

**접근 방법**:
- for 루프로 K=1~10 각각에 대해 K-Means 학습
- 각 모델의 inertia_ 값 저장
- px.line()으로 꺾은선 그래프 생성

**핵심 개념**:
- Elbow Method: Inertia 감소율이 급격히 줄어드는 지점
- "팔꿈치" 형태의 꺾이는 지점이 최적 K

**대안**:
```python
# 리스트 컴프리헨션
inertias = [KMeans(n_clusters=k, random_state=42).fit(X_scaled).inertia_ 
            for k in range(1, 11)]
```

**실무 팁**:
- Elbow가 명확하지 않을 수 있음
- Silhouette Score와 함께 사용 권장

---

## Q5. 군집 시각화 ⭐⭐

**문제**: K=4 군집 결과를 px.scatter()로 시각화하세요. (color='Cluster')

In [ ]:
# 정답 코드
# K=4 K-Means 수행
kmeans_q5 = KMeans(n_clusters=4, random_state=42, n_init=10)
labels_q5 = kmeans_q5.fit_predict(X_scaled)

# DataFrame에 군집 레이블 추가
df_q5 = df_customers.copy()
df_q5['Cluster'] = labels_q5.astype(str)  # 문자열로 변환 (범주형)

# px.scatter로 시각화
fig = px.scatter(df_q5, x='Annual_Income', y='Spending_Score',
                 color='Cluster',
                 title='K-Means 군집화 결과 (K=4)',
                 labels={'Annual_Income': '연간 소득', 'Spending_Score': '소비 점수'},
                 color_discrete_sequence=px.colors.qualitative.Set1)
fig.show()

In [ ]:
# 테스트/검증
assert 'Cluster' in df_q5.columns, "Cluster 컬럼이 있어야 합니다"
assert df_q5['Cluster'].nunique() == 4, "4개 군집이 있어야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- 군집화 결과(레이블)를 DataFrame에 추가
- color 파라미터로 군집별 색상 구분

**핵심 개념**:
- 레이블을 문자열로 변환: 범주형으로 인식
- color_discrete_sequence: 색상 팔레트 지정

**대안**:
```python
# go.Scatter로 더 세밀한 제어
fig = go.Figure()
for cluster in range(4):
    mask = labels_q5 == cluster
    fig.add_trace(go.Scatter(
        x=df_q5.loc[mask, 'Annual_Income'],
        y=df_q5.loc[mask, 'Spending_Score'],
        mode='markers', name=f'군집 {cluster}'
    ))
```

**실무 팁**:
- hover_data로 추가 정보 표시
- 중심점도 함께 표시하면 해석 용이

---

## Q6. DBSCAN 적용 ⭐⭐⭐

**문제**: eps=0.3, min_samples=5로 DBSCAN을 수행하고 군집 수와 노이즈 수를 출력하세요.

In [ ]:
# 정답 코드
from sklearn.cluster import DBSCAN

# DBSCAN 모델 생성 및 학습
dbscan_q6 = DBSCAN(eps=0.3, min_samples=5)
labels_q6 = dbscan_q6.fit_predict(X_scaled)

# 군집 수 계산 (노이즈 -1 제외)
n_clusters_q6 = len(set(labels_q6)) - (1 if -1 in labels_q6 else 0)

# 노이즈 수 계산
n_noise_q6 = (labels_q6 == -1).sum()

print(f"발견된 군집 수: {n_clusters_q6}")
print(f"노이즈(이상치) 수: {n_noise_q6}")
print(f"\n군집별 샘플 수:")
for label in sorted(set(labels_q6)):
    count = (labels_q6 == label).sum()
    name = '노이즈' if label == -1 else f'군집 {label}'
    print(f"  {name}: {count}")

In [ ]:
# 테스트/검증
assert n_clusters_q6 >= 0, "군집 수는 0 이상"
assert n_noise_q6 >= 0, "노이즈 수는 0 이상"
assert len(labels_q6) == 500, "모든 샘플이 할당되어야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- DBSCAN(eps, min_samples)로 모델 생성
- 레이블에서 -1은 노이즈
- set()으로 고유 레이블 추출 후 -1 제외

**핵심 개념**:
- eps: 이웃 반경 (epsilon)
- min_samples: 핵심점(core point)이 되기 위한 최소 이웃 수
- 레이블 -1: 어느 군집에도 속하지 않는 노이즈

**파라미터 튜닝 팁**:
- eps가 너무 작으면: 모든 점이 노이즈
- eps가 너무 크면: 모든 점이 하나의 군집
- k-distance plot으로 최적 eps 탐색

**흔한 실수**:
- 노이즈(-1)를 군집으로 카운트
- 스케일링 안 함

---

## Q7. 계층적 군집화 ⭐⭐⭐

**문제**: Ward linkage로 4개 군집의 계층적 군집화를 수행하고 Silhouette Score를 계산하세요.

In [ ]:
# 정답 코드
from sklearn.cluster import AgglomerativeClustering

# 계층적 군집화 모델 생성 및 학습
hierarchical_q7 = AgglomerativeClustering(
    n_clusters=4, 
    linkage='ward'  # Ward 방법: 분산 최소화
)
labels_q7 = hierarchical_q7.fit_predict(X_scaled)

# Silhouette Score 계산
sil_score_q7 = silhouette_score(X_scaled, labels_q7)

print(f"계층적 군집화 (Ward, K=4) Silhouette Score: {sil_score_q7:.4f}")
print(f"\n군집별 샘플 수:")
for cluster in range(4):
    count = (labels_q7 == cluster).sum()
    print(f"  군집 {cluster}: {count}개")

In [ ]:
# 테스트/검증
assert len(set(labels_q7)) == 4, "4개 군집이 있어야 합니다"
assert -1 <= sil_score_q7 <= 1, "Silhouette Score 범위 확인"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- AgglomerativeClustering으로 병합 군집화
- linkage='ward': 분산 증가를 최소화하는 방식

**핵심 개념**:
- 계층적 군집화: 상향식(Agglomerative) 또는 하향식(Divisive)
- Linkage 방법:
  - single: 최소 거리
  - complete: 최대 거리
  - average: 평균 거리
  - ward: 분산 최소화 (가장 많이 사용)

**대안**:
```python
# scipy로 dendrogram 그리기
from scipy.cluster.hierarchy import dendrogram, linkage
Z = linkage(X_scaled, method='ward')
dendrogram(Z)
```

**실무 팁**:
- 대용량 데이터에는 느림 (O(n^2) 또는 O(n^3))
- 샘플링 후 사용 권장

---

## Q8. GMM 확률 분석 ⭐⭐⭐⭐

**문제**: 4개 컴포넌트의 GMM을 학습하고, 가장 불확실한 샘플 5개(최대 확률이 가장 낮은)를 찾으세요.

In [ ]:
# 정답 코드
from sklearn.mixture import GaussianMixture

# GMM 모델 생성 및 학습
gmm_q8 = GaussianMixture(n_components=4, random_state=42)
gmm_q8.fit(X_scaled)

# 각 샘플의 군집 소속 확률
probs_q8 = gmm_q8.predict_proba(X_scaled)

# 각 샘플의 최대 확률 (가장 확신있는 군집에 속할 확률)
max_probs = probs_q8.max(axis=1)

# 최대 확률이 가장 낮은 5개 샘플 (가장 불확실한 샘플)
uncertain_indices = max_probs.argsort()[:5]

print("=== 가장 불확실한 샘플 TOP 5 ===")
for rank, idx in enumerate(uncertain_indices, 1):
    print(f"\n{rank}. 샘플 {idx}")
    print(f"   소득: {df_customers.loc[idx, 'Annual_Income']}, 소비점수: {df_customers.loc[idx, 'Spending_Score']}")
    print(f"   최대 확률: {max_probs[idx]:.4f}")
    print(f"   각 군집 확률: {probs_q8[idx].round(3)}")

In [ ]:
# 테스트/검증
assert probs_q8.shape == (500, 4), "확률 행렬 크기 확인"
assert len(uncertain_indices) == 5, "5개 샘플 추출"
assert max_probs[uncertain_indices[0]] <= max_probs[uncertain_indices[-1]], "정렬 확인"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- GMM의 predict_proba()로 소속 확률 획득
- 각 행에서 최대값 = 가장 확신 있는 군집 확률
- argsort()로 오름차순 정렬 후 앞 5개 선택

**핵심 개념**:
- GMM은 소프트 클러스터링: 확률 기반 할당
- 최대 확률이 낮음 = 경계에 있는 샘플
- 불확실한 샘플 = 여러 군집에 걸쳐 있음

**대안**:
```python
# DataFrame으로 정리
df_probs = pd.DataFrame(probs_q8, columns=[f'군집{i}' for i in range(4)])
df_probs['max_prob'] = df_probs.max(axis=1)
df_probs.nsmallest(5, 'max_prob')
```

**실무 팁**:
- 불확실한 샘플은 추가 조사 필요
- 마케팅에서 타겟팅 신중히 접근

---

## Q9. 최적 K 탐색 ⭐⭐⭐⭐

**문제**: K=2~8에 대해 Silhouette Score를 계산하고, 최적 K를 찾아 시각화하세요.

In [ ]:
# 정답 코드
# K=2~8에 대해 Silhouette Score 계산
K_range_q9 = range(2, 9)
silhouette_scores_q9 = []

for k in K_range_q9:
    kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels_temp = kmeans_temp.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels_temp)
    silhouette_scores_q9.append(score)
    print(f"K={k}: Silhouette = {score:.4f}")

# 최적 K 찾기
best_k = list(K_range_q9)[np.argmax(silhouette_scores_q9)]
best_score = max(silhouette_scores_q9)

print(f"\n최적 K: {best_k} (Silhouette = {best_score:.4f})")

In [ ]:
# 시각화
fig = px.bar(x=list(K_range_q9), y=silhouette_scores_q9,
             title='K별 Silhouette Score (높을수록 좋음)',
             labels={'x': '군집 수 (K)', 'y': 'Silhouette Score'},
             color=silhouette_scores_q9,
             color_continuous_scale='viridis')

# 최적 K 강조
fig.add_annotation(x=best_k, y=best_score,
                   text=f"최적 K={best_k}",
                   showarrow=True, arrowhead=2)
fig.show()

In [ ]:
# 테스트/검증
assert len(silhouette_scores_q9) == 7, "K=2~8 총 7개"
assert 2 <= best_k <= 8, "최적 K는 범위 내"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- for 루프로 각 K에 대해 군집화 및 평가
- argmax()로 최고 점수의 인덱스 찾기
- px.bar()로 비교 시각화

**핵심 개념**:
- Silhouette Score가 가장 높은 K가 최적
- 단, 비즈니스 해석 가능성도 고려

**대안**:
```python
# Elbow + Silhouette 동시 비교
fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=list(K_range), y=inertias), row=1, col=1)
fig.add_trace(go.Bar(x=list(K_range), y=silhouettes), row=1, col=2)
```

**실무 팁**:
- 최적 K가 여러 개일 수 있음
- 도메인 지식과 함께 결정

---

## Q10. 고객 세분화 종합 ⭐⭐⭐⭐⭐

**문제**: 
1. K-Means (K=4)로 고객을 세분화하고
2. 각 세그먼트의 평균 소득, 소비점수, 고객 수를 DataFrame으로 정리하고
3. px.scatter()로 세그먼트별 시각화를 완성하세요.

In [ ]:
# 정답 코드

# 1. K-Means (K=4)로 고객 세분화
kmeans_q10 = KMeans(n_clusters=4, random_state=42, n_init=10)
labels_q10 = kmeans_q10.fit_predict(X_scaled)

# DataFrame에 세그먼트 추가
df_q10 = df_customers.copy()
df_q10['Segment'] = labels_q10

# 2. 세그먼트별 통계 DataFrame
segment_stats = df_q10.groupby('Segment').agg({
    'Annual_Income': 'mean',
    'Spending_Score': 'mean',
    'CustomerID': 'count'
}).round(1)

segment_stats.columns = ['평균 소득', '평균 소비점수', '고객 수']

# 세그먼트 이름 부여
segment_names_q10 = {}
for idx, row in segment_stats.iterrows():
    income = '고소득' if row['평균 소득'] > 50 else '저소득'
    spending = '고소비' if row['평균 소비점수'] > 50 else '저소비'
    segment_names_q10[idx] = f"{income}-{spending}"

segment_stats['세그먼트명'] = segment_stats.index.map(segment_names_q10)

print("=== 고객 세그먼트 분석 결과 ===")
print(segment_stats)

In [ ]:
# 3. px.scatter()로 세그먼트별 시각화
df_q10['Segment_Name'] = df_q10['Segment'].map(segment_names_q10)

fig = px.scatter(df_q10, x='Annual_Income', y='Spending_Score',
                 color='Segment_Name',
                 title='고객 세분화 결과 (K-Means, K=4)',
                 labels={'Annual_Income': '연간 소득 (천만원)',
                        'Spending_Score': '소비 점수',
                        'Segment_Name': '세그먼트'},
                 hover_data=['CustomerID', 'Age'],
                 color_discrete_sequence=px.colors.qualitative.Bold,
                 opacity=0.7)

# 중심점 추가
centers_orig_q10 = scaler.inverse_transform(kmeans_q10.cluster_centers_)
fig.add_trace(go.Scatter(
    x=centers_orig_q10[:, 0], y=centers_orig_q10[:, 1],
    mode='markers+text',
    marker=dict(symbol='x', size=20, color='black', line=dict(width=2)),
    text=[f'C{i}' for i in range(4)],
    textposition='top center',
    name='중심점'
))

fig.update_layout(height=600)
fig.show()

In [ ]:
# 테스트/검증
assert 'Segment' in df_q10.columns, "Segment 컬럼 필요"
assert len(segment_stats) == 4, "4개 세그먼트"
assert '평균 소득' in segment_stats.columns, "평균 소득 컬럼 필요"
assert '평균 소비점수' in segment_stats.columns, "평균 소비점수 컬럼 필요"
assert '고객 수' in segment_stats.columns, "고객 수 컬럼 필요"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. K-Means 학습 및 레이블 할당
2. groupby()로 세그먼트별 통계 계산
3. 세그먼트 특성에 따른 이름 부여
4. px.scatter()로 색상 구분 시각화
5. 중심점 표시로 해석 용이성 향상

**핵심 개념**:
- 세분화(Segmentation): 비즈니스 전략 수립의 기초
- 프로파일링: 각 세그먼트의 특성 파악
- 시각화: 결과 커뮤니케이션의 핵심

**대안**:
```python
# 다차원 분석 (3개 이상 특성)
X_multi = df_customers[['Annual_Income', 'Spending_Score', 'Age', 'Visit_Frequency']]
X_multi_scaled = StandardScaler().fit_transform(X_multi)
# PCA로 2D 축소 후 시각화
```

**흔한 실수**:
- 스케일링 전 데이터로 군집화
- 중심점을 스케일링 역변환 안 함
- 비즈니스 해석 없이 숫자만 나열

**실무 팁**:
- 세그먼트별 마케팅 전략 수립
- 주기적으로 재군집화하여 변화 추적
- A/B 테스트로 전략 효과 검증

---

## 📊 퀴즈 난이도 분포 요약

| 난이도 | 퀴즈 번호 | 주요 개념 |
|--------|----------|----------|
| ⭐ | Q1, Q2 | K-Means 기본, Inertia |
| ⭐⭐ | Q3, Q4, Q5 | Silhouette, Elbow Plot, 시각화 |
| ⭐⭐⭐ | Q6, Q7 | DBSCAN, 계층적 군집화 |
| ⭐⭐⭐⭐ | Q8, Q9 | GMM 확률, 최적 K 탐색 |
| ⭐⭐⭐⭐⭐ | Q10 | 고객 세분화 종합 프로젝트 |

**총 10문제**: 3-3-2-2 분포 준수